Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import numba
import re
import cv2
import ast
import matplotlib.pyplot as plt

from numba import jit
from typing import List, Union, Tuple

Function to calculate IoU

In [2]:
@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area

Enter the bounding box for the ground truth (box 1) followed by the bounding box of the predicted model(box 2)

In [4]:
box1 = np.array([297,204,309.5,412.5])
box2 = np.array([0,218,309,412])

Calculate the IoU by filling the function with the bounding boxes

In [5]:
calculate_iou(box1, box2, form='coco')

0.02067243632035808

In [10]:
box1 = np.array([297,204,309.5,412.5])
box2 = np.array([100,29,309.5,412.5])

In [11]:
calculate_iou(box1, box2, form='coco')

0.11784122630883001

In [8]:
box1 = np.array([297,204,309.5,412.5])
box2 = np.array([279,134,309.5,412.5])

In [9]:
calculate_iou(box1, box2, form='coco')

0.6427861985576715